# Simple plots
after running the verification experiment tutorial_barotropic_gyre in "myrun" with 
 useSingleCPUio = .TRUE.,
 dumpInitAndLast = .FALSE.,
 monitorFreq = 864000.,
 dumpFreq = 8640000.,
and for 1000 days (72000 timesteps), run this notebook to plot sea surface height "eta".
You can also modify the parameters in the namelist file to reduce the amount of lateral friction and rerun the model.

In [2]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
import os
# helper function
def sq(a):
    a = np.squeeze(a)
    return np.ma.masked_array(a==0.,a)

bdir = '/Users/mlosch/Documents/teaching/GLOMAR/AdvancedOceanModelling/MITgcm/verification'
experiment0 = 'tutorial_barotropic_gyre/run'
experiment1 = 'tutorial_barotropic_gyre/run_noadv'

either read with MITgcmutils and plot with matplotlib functions ...

In [3]:
from MITgcmutils import rdmds

m2km=1e-3
xg = rdmds(os.path.join(bdir,experiment0,'XG'))*m2km
yg = rdmds(os.path.join(bdir,experiment0,'YG'))*m2km
xc = rdmds(os.path.join(bdir,experiment0,'XC'))*m2km
yc = rdmds(os.path.join(bdir,experiment0,'YC'))*m2km

eta0 = rdmds(os.path.join(bdir,experiment0,'Eta'),np.NaN)
eta1 = rdmds(os.path.join(bdir,experiment1,'Eta'),np.NaN)

plot last output and compare experiment 0 and 1

In [4]:
mytime = -1
mycmap = plt.cm.viridis
plt.close('all')
fig1,ax1 = plt.subplots(1,2,sharex=True,sharey=True,figsize=(8,3.5))
csf = ax1[0].contourf(xc,yc,eta0[mytime,:,:],cmap=mycmap,levels=np.linspace(-0.06,0.06,21))
csf = ax1[1].contourf(xc,yc,eta1[mytime,:,:],cmap=mycmap,levels=np.linspace(-0.06,0.06,21))
# alternatively as pcolormesh plots
#csf = ax1[0].pcolormesh(xg,yg,eta0[mytime,:,:],cmap=mycmap,vmin=-0.05,vmax=0.05)
#csf = ax1[1].pcolormesh(xg,yg,eta1[mytime,:,:],cmap=mycmap,vmin=-0.05,vmax=0.05)
ax1[0].set_title('exp 0')
ax1[1].set_title('exp 1')
ax1[0].set_ylabel('Y (km)')
ax1[0].set_xlabel('X (km)')
ax1[1].set_xlabel('X (km)')

plt.colorbar(csf,ax=ax1[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

... or read with xmitgcm and plot using xarray methods

In [5]:
from xmitgcm import open_mdsdataset

ds = 0.
import warnings
warnings.filterwarnings('ignore')
# suppress numerous warnings only when reading data into the workspace with xmitgcm.open_mdsdataset
with warnings.catch_warnings():
    ds = open_mdsdataset(os.path.join(bdir,experiment0), prefix = 'Eta', geometry='cartesian')
    # this only works if you remove all files with timestep number zero, like : rm *.0000000000.??ta
    #ds = open_mdsdataset(os.path.join(bdir,experiment1), geometry='cartesian')


#ds.Eta.isel(time=mytime).plot.contourf(ax=ax1[0],cmap=mycmap,levels=np.linspace(-0.06,0.06,21))

ds.Eta.isel(time=mytime).plot.pcolormesh(ax=ax1[1],cmap=mycmap,levels=np.linspace(-0.06,0.06,21))

plt.tight_layout()

# if you prefer contour plots:

In [6]:
#plt.close('all')
#ds.Eta.plot.contourf(cmap=plt.cm.viridis,x="XC", y="YC", col="time", col_wrap=5,
#                                               levels=np.linspace(-0.06,0.06,21))

In [7]:
from matplotlib.animation import FuncAnimation

fig2,ax2=plt.subplots(1,1)

e = ds.Eta.where(ds.Eta!=0)
time=ds.time
mylevs=np.linspace(-0.04,0.08,21)
t=1
csf=e.isel(time=t).plot.contourf(ax=ax2,cmap=plt.cm.viridis,levels=mylevs)
#ax2.title.set_text('iter = %i, time = %s'%(e.iter[t],np.datetime_as_string(e.time[t], unit='s')))
#print(np.datetime_as_string(time[t], unit='s'))
ax2.title.set_text('iter = %i'%(e.iter[t]))
plt.tight_layout()
    
def animate(t):
    ax2.cla()
    ax2.contourf(ds.XC,ds.YC,e.isel(time=t).data, cmap=plt.cm.viridis, levels=mylevs)
#    ax2.title.set_text('iter = %i, time = %s'%(e.iter[t],np.datetime_as_string(e.time[t], unit='s')))
    ax2.title.set_text('iter = %i'%(e.iter[t]))
    plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
anim = FuncAnimation(
    fig2, animate, interval=100, frames=len(e.time)-1)
 
anim.save('animation.mp4')
#fig2.show()